# Information Extraction - Assignment
This assignment is based on the Information Extraction lecture and the lab.


Assignment2
Niranjanadevi Janagaraj
18230368

In [0]:
#Importing Library
import nltk
import re
from statistics import mode
nltk.download('all')

In [0]:
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import itertools
import json
from nltk.tag.stanford import StanfordNERTagger
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from difflib import SequenceMatcher

In [0]:
#Loading File
inputfile='football_player.txt' 
buf=open(inputfile, encoding="UTF-8")
document_list=buf.read().split('\n')
print(document_list)

In [0]:
l = []
for i in document_list:
   if len(i) != 0:
       l.append(i)
document_list = l
print(document_list)

# Task 1 (10 Marks)
Write a function that takes each document and performs:
1) sen segmentation 2) tokenization 3) part-of-speech tagging

Please keep in mind that the expected output is a list within a list as shown below.


In [0]:
def ie_preprocess(document):
  #code goes here
  sen = sent_tokenize(document)
  sen_pos = []
  pos_sent=[]
  for i in sen:
    text = word_tokenize(i)
    sen_pos.append(nltk.pos_tag(text))
  return sen_pos
     

Run the following code to check your result for the first document (Ronaldo).

In [0]:
first_doc=document_list[0]
pos_sent=ie_preprocess(first_doc)
pos_sent

Expected output
 [...[('He', 'PRP'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('forward', 'NN'),
  ('and', 'CC'),
  ('serves', 'NNS'),
  ('as', 'IN'),
  ('captain', 'NN'),
  ('for', 'IN'),
  ('Portugal', 'NNP'),
  ('.', '.')], ...]

# Task 2 (20 Marks)
Write a function that will take the list of tokens with POS tags for each sen and returns the named entities (NE). 

Hint: Use binary=True while calling NE chunk function

In [0]:
entities=[]
def named_entity_finding(pos_sent):
  tree = nltk.ne_chunk(pos_sent,binary=True)
  for subtree in tree.subtrees():
    if subtree.label() == 'NE':
      entity = ""
      for leaf in subtree.leaves():
        entity = entity + leaf[0] + " "
      entities.append(entity.strip())
  return entities

In [0]:
#pos_sent=ie_preprocess(document_list[0])
named_entity_finding(pos_sent[0])

Expected output ['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

# Task 3 (10 Marks)

Now use the named_entity_finding() function to extract all NEs for each document.

Hint: pos_sents holds the list of lists of tokens with POS tags

In [0]:
def NE_flat_list_fn(pos_sent):
  NE=[]
  for i in pos_sent:
    new_pos = ie_preprocess(i)
    for pos in new_pos:
      entity = named_entity_finding(pos)
      NE.append(entity)
      NE_flat_list = list(itertools.chain.from_iterable(NE))
  return NE_flat_list

new=NE_flat_list_fn(document_list)

# Task 4 (40 Marks)

Write functions to extract the name of the player, country of origin and date of birth as well as the following relations: team(s) of the player and position(s) of the player.

Hint: Use the re.compile() function to create the extraction patterns

Reference: https://docs.python.org/3/howto/regex.html

In [0]:
def name_of_the_player(doc):
  sen = sent_tokenize(doc)[0]
  name=re.compile(r'^(\w.+)(?=\s+.born)')
  name_player=name.match(sen)[0].split(",")[0]
  return name_player

def country_of_origin(doc):
  country=[]
  origin=re.compile(r'((?:[\S,]+\s+){0,1})national team')
  for i, sent in enumerate(sent_tokenize(doc)):
    if i != 2:
      team = origin.findall(sent)
      if len(team) != 0:
        country.append(team[0])
  return country

def date_of_birth(doc):
  born=[]
  sen = sent_tokenize(doc)[0]
  Date_of_birth = re.compile(r'born\b\s*((?:\S+\s+){0,3})')
  birth = Date_of_birth.findall(sen)[0]
  birth = re.sub('\W+',' ', birth )
  return birth


    
def position_of_the_player(doc):
  position = ["forward", "captain", "attacking midfielder", "striker", "winger", "central midfielder", "defensive tackle", "defensive end"]
  position_of_player=[]
  player=[]
  sent = sent_tokenize(doc)
  for i, sent in enumerate(sent):
    for j in position:
      position_of_player = re.compile(r'\b({0})\b'.format(j))
      ans = bool(position_of_player.search(sent))
      if ans == True:
        player.append(j)
  return list(set(player))
    

In [0]:
def team_of_the_player(doc):
  sen = sent_tokenize(doc)
  new = []
  team_of_name = []
  national = []
  named=[]
  
  pos_sent = ie_preprocess(doc)
  for i, sent in enumerate(sen):
    if i == 2:
      break
      
      
    match = re.compile(r'((?:[\S,]+\s+){0,1})national team')       
    team_player = re.compile(r'club\s+((?:[\S,]+\s*){0,2})')       
    team = match.findall(sent)
    if len(team_player .findall(sent)) != 0:
      for f in team_player .findall(sent):
        national.append(f)
        
    tree = nltk.ne_chunk(pos_sent[i],binary=False)                 
    for subtree in tree.subtrees():
      if subtree.label() == 'ORGANIZATION':
        entity = ""
        for leaf in subtree.leaves():
          entity = entity + leaf[0] + " "
        named.append(entity.strip())
    team_of_name.append(named)                                      
    if len(team) != 0:
      l = team[0]+"national team"
      new.append(l) 
      
  if len(list(set(new))) != 0:
    national_team = list(set(new))[0]
  else:
     national_team = country_of_origin(doc) 
  team_of_name = list(itertools.chain.from_iterable(team_of_name))
  
  
  new_club = []
  for i in national:
    new_club.extend(nltk.word_tokenize(i))                       
  
  for i, s in enumerate(national):
    national[i] = national[i].rstrip()
  if len(list(set(team_of_name).intersection(national))) != 0:
    team_of_name = list(set(team_of_name).intersection(national))   
  
  if len(list(set(new))) == 0:
    team_of_name.append(national_team[0]+ " national team")       
  else:
    team_of_name.append(national_team)
  team_of_name = list(set(team_of_name).difference(nltk.word_tokenize(sen[0])[0:6]))

  return team_of_name

In [31]:
#Printing and checking answers

print(name_of_the_player(document_list[0]))
print(country_of_origin(document_list[1]))
print(date_of_birth(document_list[1]))
print(team_of_the_player(document_list[7]))
print(position_of_the_player(document_list[1]))

Cristiano Ronaldo dos Santos Aveiro
['Argentina ']
24 June 1987 
['Arsenal', 'German national team']
['captain', 'forward']


Execute the below command to check your fuction


Expected output '5 February 1992'

# Task 5 (10 Marks)

Write a function using the outputs from the previous functions to generate JSON-LD output as follows.

Reference: https://json-ld.org/primer/latest/

{ "@id": "http://my-soccer-ontology.com/footballer/name_of_the_player",

    "name": "",
    "born": "",
    "country": "",
    "position": [
        { "@id": "http://my-soccer-ontology.com/position",
            "type": ""
        }
     ]   
     "team": [
        { "@id": "http://my-soccer-ontology.com/team",
            "name": ""
        }   
     ]
}


In [0]:
def generate_jsonld(arg):
  soccerld = { "@id": "http://my-soccer-ontology.com/footballer/"+arg[0],

           "name": arg[0],
           "born": arg[1],
           "country": arg[2],
           "position": [
               { "@id": "http://my-soccer-ontology.com/position/",
                   "type": arg[3]
               }
            ],   
            "team": [
               { "@id": "http://my-soccer-ontology.com/team/",
                   "name": arg[4]
               }   
            ]
       }

  return json.dumps(soccerld)

In [33]:
def json_function(doc):
  arguments = [name_of_the_player(doc), date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), team_of_the_player(doc)]
  return arguments

arguments=json_function(document_list[0])
generate_jsonld(arguments)

'{"@id": "http://my-soccer-ontology.com/footballer/Cristiano Ronaldo dos Santos Aveiro", "name": "Cristiano Ronaldo dos Santos Aveiro", "born": "5 February 1985 ", "country": ["Portugal "], "position": [{"@id": "http://my-soccer-ontology.com/position/", "type": ["captain", "forward"]}], "team": [{"@id": "http://my-soccer-ontology.com/team/", "name": ["Real Madrid", "Portugal national team"]}]}'

# Task 6 (10 Marks)
Identify one other relation (besides team and player) and write a function to extract this. Also extend the JSON-LD output accordingly.